
# 🧭 Agentic AI Project – Google Colab Setup Guide

This notebook lets students easily upload and run the **Agentic AI project** on Google Colab.

### 🚀 What it does
1. Upload your local zipped project folder (e.g. `carbon-aware-agent-workshop-colab.zip`)
2. Install required dependencies
3. Set environment variables - specify your API key
4. Run the sample solution - chatbot
5. Modify the student tools.py
6. Run the student - cahtbot


# Step 1: **Download** **carbon-aware-agent-workshop-colab.zip**;

# Run below cell, upon prompt to **upload** file, select **carbon-aware-agent-workshop-colab.zip** and upload


In [ ]:
from google.colab import files
import zipfile, io, os

uploaded = files.upload()  # Choose your .zip file when prompted

# Extract contents
zip_name = next(iter(uploaded))
zf = zipfile.ZipFile(io.BytesIO(uploaded[zip_name]))
zf.extractall(".")

# Confirm extracted structure
!ls -R | head -n 50
print("\n✅ Files extracted successfully!")

# Step 2: Install required dependencies

In [ ]:
%pip install -q langchain-openai langgraph python-dotenv python-dateutil

print("✅ Dependencies installed successfully!")

# Step 3: Set environment variables. Specify your API key

In [ ]:
import os

# Make sure BASE_DIR points to the project root (important for memory/data paths)
os.environ["BASE_DIR"] = "/content/carbon-aware-agent-workshop-colab"
os.environ["OPENAI_API_KEY"] = "YOUR-API-KEY"
os.environ["OPENAI_MODEL"] = "gpt-4o-mini"

print("BASE_DIR:", os.environ["BASE_DIR"])

# Step 4: Run the sample solution - chatbot

# Step 4.1: Build app and helper

In [ ]:
# Change to the folder where the code actually lives after unzip
%cd /content/carbon-aware-agent-workshop-colab/solution/src
print("Working directory:", os.getcwd())


from run_chat import build_app, SYSTEM
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage,ToolMessage


app = build_app()
state = {"messages": [SystemMessage(content=SYSTEM)]}


def color(s, c):
    codes = dict(gray=90, red=91, green=92, yellow=93, blue=94, magenta=95, cyan=96, reset=0)
    return f"\033[{codes[c]}m{s}\033[{codes['reset']}m"

def print_debug(out):
    print(color("\n[DEBUG] Messages this turn:", "green"))
    #print(out["messages"])
    for i, m in enumerate(out["messages"], 1):
        t = type(m).__name__
        content = (getattr(m, "content", "") or "")
        print(color(f"{i:02d}. {t}:", "green"), repr(content[:100]))

        tool_calls = getattr(m, "tool_calls", None) or getattr(m, "additional_kwargs", {}).get("tool_calls", [])
        for c in tool_calls or []:
            fn = c.get("name")
            args = c.get("args")
            print(color(f"    └─ tool_call: {fn}({args})", "green"))

        if isinstance(m, ToolMessage):
            print(color(f"    └─ tool_result from {m.name}: {m.content[:120]}", "green"))

def turn(user_text: str) -> str:
    state["messages"].append(HumanMessage(content=user_text))
    out = app.invoke(state, config={"recursion_limit": 100})
    state.update(out)

    print_debug(out)

    # --- Return the assistant's last text message ---
    for m in reversed(out["messages"]):
        if isinstance(m, AIMessage) and (m.content or "").strip():
            return m.content
    return "(no reply)"

# Step 4.2: Interactive chat loop

In [ ]:
print("Agent ready. Try e.g.:")
print(" • I want to schedule a job")
print(" • tomorrow 10am")
print(" • I prefer regions SG, EU_WEST")
print("Type /reset to clear, /exit to quit.\n")


try:
    while True:
        user = input("You: ").strip()
        if not user:
            continue
        if user.lower() in {"/exit", "exit", "quit"}:
            print("Bye!")
            break
        if user.lower() in {"/reset", "reset"}:
            state = {"messages": [SystemMessage(content=SYSTEM)]}
            print("State reset.\n")
            continue
        reply = turn(user)
        print("\nAgent:", reply, "\n")
except KeyboardInterrupt:
    print("\nStopped.")

# Step 5: modify **student->src->tools.py** to complete **recommend_best_tool** function and save

# Step 6: Test Student Project

In [6]:
# Change to the folder where the code actually lives after unzip
%cd /content/carbon-aware-agent-workshop-colab/student/src
print("Working directory:", os.getcwd())

from langchain_core.messages import SystemMessage, HumanMessage, AIMessage

def reload_src():
    import sys, importlib
    sys.path.insert(0, '/content/carbon-aware-agent-workshop-colab/student/src')
    for m in ['tools', 'run_chat']:
        sys.modules.pop(m, None)
    importlib.invalidate_caches()
    import run_chat
    return importlib.reload(run_chat)

rc = reload_src()
app = rc.build_app()

state = {"messages": [SystemMessage(content=SYSTEM)]}

def turn(user_text: str) -> str:
    state["messages"].append(HumanMessage(content=user_text))
    out = app.invoke(state, config={"recursion_limit": 100})
    state.update(out)

    print_debug(out)

    for m in reversed(out["messages"]):
        if isinstance(m, AIMessage) and (m.content or "").strip():
            return m.content
    return "(no reply)"

print("Agent ready. Try e.g.:")
print(" • I want to schedule a job")
print(" • tomorrow 10am")
print(" • I prefer regions SG, EU_WEST")
print("Type /reset to clear, /exit to quit.\n")

try:
    while True:
        user = input("You: ").strip()
        if not user:
            continue
        if user.lower() in {"/exit", "exit", "quit"}:
            print("Bye!")
            break
        if user.lower() in {"/reset", "reset"}:
            state = {"messages": [SystemMessage(content=SYSTEM)]}
            print("State reset.\n")
            continue
        reply = turn(user)
        print("Agent:", reply, "\n")
except KeyboardInterrupt:
    print("\nStopped.")


/content/carbon-aware-agent-workshop-colab/student/src
Working directory: /content/carbon-aware-agent-workshop-colab/student/src
Agent ready. Try e.g.:
 • I want to schedule a job
 • tomorrow 10am
 • I prefer regions SG, EU_WEST
Type /reset to clear, /exit to quit.

State reset.


[DEBUG] Messages this turn:
01. SystemMessage: 'You are a single-agent carbon-aware scheduler.\n- Goal: recommend the best (region, start time) aroun'
02. HumanMessage: 'what is the date today?'
03. AIMessage: "Today's date is October 24, 2023."
Agent: Today's date is October 24, 2023. 


[DEBUG] Messages this turn:
01. SystemMessage: 'You are a single-agent carbon-aware scheduler.\n- Goal: recommend the best (region, start time) aroun'
02. HumanMessage: 'what is the date today?'
03. AIMessage: "Today's date is October 24, 2023."
04. HumanMessage: 'fetch the latest date from web'
05. AIMessage: "I'm unable to fetch real-time data from the web. However, I can confirm that today's date is October"
Agent: I'm un